<a href="https://colab.research.google.com/github/ZhangShuui/SemEval-2025-admire-Task1/blob/main/admire_experiments_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup and Preprocessing

In [5]:
# install clip
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git

import clip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00


In [6]:
!pip install anyio==3.5.0 openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/16

In [7]:
# for downloading the train zip from Drive
!pip install -q gdown

import gdown

# download taskA file from Adam's Drive (public) and unzip
file_id = "105JdQU_u98w_xSYaNNSj-r4RsyTPXZEF"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "taskA.zip", quiet=True)
! unzip -q - taskA.zip

In [8]:
import os
import pandas as pd

# define locations
taska_folder = "train"
taska_tsv_filename = "subtask_a_train.tsv"

# load data
df = pd.read_csv(f"{taska_folder}/{taska_tsv_filename}", delimiter="\t")

# fix incorrect row in dataset
df.loc[df['compound'] == "pain in the neck", 'sentence_type'] = 'literal'

In [9]:
from ast import literal_eval

# Preprocess dataframe (image paths, etc.)
image_name_cols = ['image1_name', 'image2_name', 'image3_name', 'image4_name', 'image5_name']
df['image_paths'] = df.apply(lambda row: [os.path.join(taska_folder, row['compound'].replace("'", "_"), row[image_name]) for image_name in image_name_cols], axis=1)
df['image_idx_map'] = df.apply(lambda row: {row[name]: i for i, name in enumerate(image_name_cols)}, axis=1)
df['expected_order_indices'] = df.apply(lambda row: [row['image_idx_map'][name] for name in literal_eval(row['expected_order'])], axis=1)

### Model-specific functions

In [10]:
import torch
from PIL import Image
from ast import literal_eval

def get_image_ranking_clip(model, image_processor, image_paths, sentence):
    image_inputs = torch.stack([image_processor(Image.open(ipath)) for ipath in image_paths]).to(device)
    text_input = clip.tokenize(sentence).to(device)

    with torch.no_grad():
        # compute embeddings
        image_features = model.encode_image(image_inputs)
        text_features = model.encode_text(text_input)

    # normalize features
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # compute similarity scores
    similarity = (100.0 * text_features @ image_features.T).softmax(dim=-1)

    # rank images by similarity
    probs, indices = similarity[0].topk(5)
    return probs, indices

In [11]:
def get_image_ranking_align(model, processor, image_paths, sentence):
    image_inputs = [Image.open(ipath) for ipath in image_paths]
    inputs = processor(images=image_inputs ,text=sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits_per_text = outputs.logits_per_text[0]
    probs = logits_per_text.softmax(dim=-1)
    ids_sorted = torch.argsort(probs, descending=True)
    return probs[ids_sorted], ids_sorted

In [12]:
!pip install open_clip_torch
import open_clip

def openclip_image_ranking(model, image_processor, tokenizer, image_paths, sentence):
    image_inputs = torch.stack([preprocess_openclip(Image.open(ipath)) for ipath in image_paths]).to(device)
    text_input = tokenizer([sentence]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_inputs)
        text_features = model.encode_text(text_input)

    # normalise features
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # dot product & softmax
    similarity = (100.0 * text_features @ image_features.T).softmax(dim=-1)

    # order by similarity
    probs, indices = similarity[0].topk(5)
    return probs, indices

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.7 MB/s eta 0:00:00


### General functions for experiment execution

In [13]:
def get_predictions(model, processor, image_paths_list, text_inputs, base_model, model_name):
    """
    Uses get_image_ranking to generate predictions and confidence scores for a image-list, text-input pairs
    """
    predictions, confidence_scores = [], []

    if base_model == "open_clip":
      tokenizer = open_clip.get_tokenizer(model_name)

    for ipaths, text in zip(image_paths_list, text_inputs):
        if len(ipaths) == 0:
            predictions.append([])
            confidence_scores.append([])
            continue

        # values, indices = get_image_ranking(ipaths, text)
        if base_model == "CLIP":
          values, indices = get_image_ranking_clip(model, processor, ipaths, text)
        elif base_model == "Align":
          values, indices = get_image_ranking_align(model, processor, ipaths, text)
        elif base_model == "open_clip":
          # values, indices = get_image_ranking_open_clip(model, processor, ipaths, text, model_name)
          values, indices = openclip_image_ranking(model, processor, tokenizer, ipaths, text)
        # elif base_model == "BLIP":
        #   values, indices = get_image_ranking_blip(model, image_processor, ipaths, text)
        else:
          raise ValueError(f"Unknown base_model: {base_model}")
        predictions.append(list(indices.cpu()))
        confidence_scores.append(100 * values)

    return predictions, confidence_scores


In [14]:
from scipy.stats import spearmanr

def evaluate_predictions(predictions, df, weights=[0.4, 0.3, 0.2, 0.1, 0.0]):
    """
    Takes predictions, returns three types of evaluation metrics:
    - Top-1 Accuracy
    - Average Spearman Correlation
    - Average Weighted Accuracy
    """
    correct_top1 = 0
    spearman_scores, weighted_scores = [], []

    for i in range(len(predictions)):
        if len(predictions[i]) == 0:
            continue

        # Ground truth and predictions
        pred_order = [df['image_idx_map'].iloc[i][os.path.basename(df['image_paths'].iloc[i][j])] for j in predictions[i]]
        ground_truth_order = df['expected_order_indices'].iloc[i]

        # Top-1 accuracy
        if pred_order[0] == ground_truth_order[0]:
            correct_top1 += 1

        # Spearman correlation
        score, _ = spearmanr(pred_order, ground_truth_order)
        spearman_scores.append(score)

        # Weighted accuracy
        weighted_score = sum(weights[j] for j, img in enumerate(pred_order) if img == ground_truth_order[j])
        weighted_scores.append(weighted_score)

    return {
        "top1_accuracy": correct_top1 / len(predictions),
        "average_spearman": sum(spearman_scores) / len(spearman_scores),
        "average_weighted_accuracy": sum(weighted_scores) / len(weighted_scores),
        "spearman_scores": spearman_scores,
        "weighted_scores": weighted_scores
    }


In [15]:
import csv

def save_results(experiment_name, base_model, model_name, metrics, results_file="experiment_results.csv"):
    """
    Save experiment results to a CSV file.
    """
    # Add experiment name to metrics
    results_row = {
        "base_model": base_model,
        "model": model_name,
        "experiment": experiment_name,
        "top1_accuracy": metrics["top1_accuracy"],
        "average_spearman": metrics["average_spearman"],
        "average_weighted_accuracy": metrics["average_weighted_accuracy"],
    }

    # Write results to CSV
    write_header = not os.path.exists(results_file)
    with open(results_file, mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=results_row.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(results_row)

    print(f"Results saved to {results_file}")

In [16]:
def run_experiment(model, processor, df, image_paths, text_inputs, model_name, experiment_name, base_model):
    """
    Run an experiment using transformed embeddings, evaluate metrics, and print results.
    """
    predictions, confidence_scores = get_predictions(model, processor, image_paths, text_inputs, base_model, model_name)
    metrics = evaluate_predictions(predictions, df)
    save_results(experiment_name, base_model, model_name, metrics)

    print(f"Top-1 Accuracy: {metrics['top1_accuracy'] * 100:.2f}%")
    print(f"Average Spearman Correlation: {metrics['average_spearman']:.2f}")
    print(f"Average Weighted Accuracy: {metrics['average_weighted_accuracy']:.2f}")

    return metrics, predictions, confidence_scores

In [17]:
import re

def save_predictions(df, image_paths, predictions, confidence_scores, metrics, prefix, preds_dir='predictions'):
    """
    Save detailed predictions and confidence scores for each example.
    """
    # create 'preds' directory if doesn't exist
    if not os.path.exists(preds_dir):
        os.makedirs(preds_dir)

    # generate output filename
    prefix = prefix.strip().replace(" ", "_")
    prefix = re.sub(r'[^a-zA-Z0-9_-]', '', prefix)
    output_path = f"{preds_dir}/{prefix}_preds.csv"

    spearman_scores = metrics["spearman_scores"]
    weighted_scores = metrics["weighted_scores"]
    with open(output_path, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["index", "compound", "ground_truth_order", "predicted_order", "top1_score", "spearman_score", "weighted_score", "confidence_scores"])

        for i, (pred, conf) in enumerate(zip(predictions, confidence_scores)):
            pred_order = [df['image_idx_map'].iloc[i][os.path.basename(image_paths.iloc[i][j])] for j in pred]
            ground_truth_order = df["expected_order_indices"].iloc[i]
            top1_score = 1 if pred_order[0] == ground_truth_order[0] else 0
            spearman_score = round(spearman_scores[i], 3)
            weighted_score = round(weighted_scores[i], 3)
            formatted_conf_scores = [round(c.item(), 3) for c in conf]
            writer.writerow([i, df["compound"].iloc[i], ground_truth_order, pred_order, top1_score, spearman_score, weighted_score, formatted_conf_scores])

    print(f"Predictions saved to {output_path}")


In [18]:
def save_predictions_to_df(df, image_paths, predictions, confidence_scores, metrics, combined_predictions, base_model, model_name, experiment_name):
    """
    Add predictions to a combined_predictions list instead of writing to a file.
    """
    spearman_scores = metrics["spearman_scores"]
    weighted_scores = metrics["weighted_scores"]

    for i, (pred, conf) in enumerate(zip(predictions, confidence_scores)):
        pred_order = [df['image_idx_map'].iloc[i][os.path.basename(image_paths.iloc[i][j])] for j in pred]
        ground_truth_order = df["expected_order_indices"].iloc[i]
        top1_score = 1 if pred_order[0] == ground_truth_order[0] else 0
        spearman_score = round(spearman_scores[i], 3)
        weighted_score = round(weighted_scores[i], 3)
        formatted_conf_scores = [round(c.item(), 3) for c in conf]

        # Add the row to the combined_predictions list
        combined_predictions.append({
            "index": i,
            "compound": df["compound"].iloc[i],
            "sentence": df["sentence"].iloc[i],
            "ground_truth_order": ground_truth_order,
            "predicted_order": pred_order,
            "top1_score": top1_score,
            "spearman_score": spearman_score,
            "weighted_score": weighted_score,
            "confidence_scores": formatted_conf_scores,
            "base_model": base_model,
            "model_name": model_name,
            "experiment": experiment_name
        })

    print(f"Predictions added to combined DataFrame for model: {model_name}, experiment: {experiment_name}")


### Prompt GPT & populate data columns

#### GPT Prompt 1: definition of compound

In [21]:
# GPT Prompt #1

import json
from google.colab import userdata
from openai import OpenAI

# initialize openai client with colab secret key
# client = OpenAI(api_key=userdata.get('OPENAI_PROJECT_KEY'))
client = OpenAI(api_key='sk-IQWpWGw3rqw1OEv102Cb0BHqya3ysEI9KKzxXNIdt3oDssnL',base_url="https://api.aiproxy.io/v1")

def generate_paraphrases_batched(compounds, sentence_types):
    """
    Generate paraphrases in batches using GPT-4.
    """
    # Filter out literals since they don't need processing
    input_data = [
        nc for nc, sentence_type in zip(compounds, sentence_types) if sentence_type != "literal"
    ]
    # Skip batch if all are literal
    if not input_data:
        return {nc: nc for nc in compounds}  # Return original NCs for all literals

    # Create a combined prompt
    examples = "\n".join([
        f'The idiom is: "{nc}".' for nc in input_data
    ])

    prompt = f"""
You are a linguistics expert specializing in idioms. For each of the idioms below, provide a definition that balances accuracy with its emotional and cultural essence.

Example:
Idiom: "cold turkey"
Definition: "Abruptly quitting a habit, marked by discomfort and determination."

Idioms to define:
{examples}

Respond in this format:
{{"idioms": [
    {{"idiom": "cold turkey", "definition": "Abruptly quitting a habit, marked by discomfort and determination."}},
    {{"idiom": "another idiom", "definition": "Its definition here."}}
]}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
        )
        content = json.loads(response.choices[0].message.content.strip())
        results = {item["idiom"]: item["definition"] for item in content["idioms"]}
        for idiom, definition in results.items():
            print(f"Idiom: {idiom}\nDefinition: {definition}\n")
        return results
    except Exception as e:
        print(f"Error generating paraphrases: {e}")
        return {}

In [22]:
# Apply to the DataFrame in batches
batch_size = 10
results = {}
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    compounds = batch['compound'].tolist()
    sentence_types = batch['sentence_type'].tolist()
    batch_results = generate_paraphrases_batched(compounds, sentence_types)
    results.update(batch_results)

# Map results back to DataFrame
df['gpt_prompt_1_resp'] = df['compound'].map(results)

# For rows with null 'paraphrased_nc', set equal to 'compound'
df['gpt_prompt_1_resp'].fillna(df['compound'], inplace=True)

Idiom: elbow grease
Definition: Significant physical effort applied to a task, often resulting in hard work and satisfaction.

Idiom: night owl
Definition: A person who is naturally more active and alert during the late hours of the night, often associated with creativity and independence.

Idiom: heart of gold
Definition: A person who is exceptionally kind, generous, and compassionate, radiating warmth and positivity.

Idiom: agony aunt
Definition: A person, typically a columnist, who offers advice on personal problems, embodying empathy and understanding.

Idiom: shrinking violet
Definition: A person who is shy or timid, often seen as delicate and gentle, avoiding the spotlight.

Idiom: banana republic
Definition: A politically unstable country with an economy dependent on a single export, often characterized by corruption and lack of governance.

Idiom: private eye
Definition: A private investigator, often engaging in clandestine activities to uncover secrets and solve mysteries, em

<ipython-input-22-36673918767d>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['gpt_prompt_1_resp'].fillna(df['compound'], inplace=True)


#### GPT Prompt 2: definition of compound with multi-step reasoning

In [23]:
# GPT Prompt #2

import json
from google.colab import userdata
from openai import OpenAI

# initialize openai client with colab secret key
# client = OpenAI(api_key=userdata.get('OPENAI_PROJECT_KEY'))
client = OpenAI(api_key='sk-IQWpWGw3rqw1OEv102Cb0BHqya3ysEI9KKzxXNIdt3oDssnL',base_url="https://api.aiproxy.io/v1")

def generate_paraphrases_batched_2(compounds, sentence_types):
    """
    Generate paraphrases in batches using GPT-4.
    """
    # Filter out literals since they don't need processing
    input_data = [
        nc for nc, sentence_type in zip(compounds, sentence_types) if sentence_type != "literal"
    ]
    # Skip batch if all are literal
    if not input_data:
        return {nc: nc for nc in compounds}  # Return original NCs for all literals

    # Create a combined prompt
    examples = "\n".join([
        f'The idiom is: "{nc}".' for nc in input_data
    ])

    prompt = f"""
You are a linguistics expert specializing in idioms. For each of the idioms below, do the following steps aloud (in writing):
1. Give a verbose explanation of the idiom, including what connotations it carries or undertones it evokes.
2. List three potential definitions, no longer than 20 words each, that capture the essence of the phrase in a general manner.
3. Choose the best definition.

Example #1:
Idiom: "cold turkey"
Definition: "Abruptly quitting a habit or addiction, overcoming discomfort or pain, requiring determination."

Example #2:
Idiom: "piece of cake"
Definition: "Easy, simple to accomplish, requiring little effort, not a problem."

Idioms to define:
{examples}

Respond in this format:
{{"idioms": [
    {{"idiom": "cold turkey", "verbose_definition": "<verbose_definition>", "possible_definitions": "<three possible definitions>", "definition": "Abruptly quitting a habit, marked by discomfort and determination."}},
    {{"idiom": "<another idiom>", "verbose_definition": "<verbose_definition>", "possible_definitions": "<three possible definitions>" , "definition": "<its definition>" }}
]}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
        )
        content = json.loads(response.choices[0].message.content.strip())
        results = {item["idiom"]: item["definition"] for item in content["idioms"]}
        for item in content["idioms"]:
            print(f"Idiom: {item['idiom']}\nVerbose: {item['verbose_definition']}\nPossible: {item['possible_definitions']}\nDefinition: {item['definition']}\n")
        return results
    except Exception as e:
        print(f"Error generating paraphrases: {e}")
        return {}

In [24]:
# Apply to the DataFrame in batches
batch_size = 10
results = {}
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    compounds = batch['compound'].tolist()
    sentence_types = batch['sentence_type'].tolist()
    batch_results = generate_paraphrases_batched_2(compounds, sentence_types)
    results.update(batch_results)

# Map results back to DataFrame
df['gpt_prompt_2_resp'] = df['compound'].map(results)

# For rows with null 'paraphrased_nc', set equal to 'compound'
df['gpt_prompt_2_resp'].fillna(df['compound'], inplace=True)

Idiom: elbow grease
Verbose: The idiom 'elbow grease' refers to the physical effort and hard work one puts into a task, often requiring sweat and determination to achieve a desired outcome. It suggests that success is not just about intellect or strategy, but more about the willingness to exert oneself physically. The connotation of the phrase evokes a sense of laborious effort, often in manual work or cleaning, indicating that true results come from hard work rather than shortcuts or easy methods.
Possible: 1. Physical effort required to accomplish a task. 2. Hard work and determination leading to results. 3. Manual labor bringing about significant change.
Definition: Physical effort and hard work needed to accomplish a task.

Idiom: night owl
Verbose: The term 'night owl' describes a person who tends to stay up late into the night and prefers to be active during these hours. Night owls typically have a different circadian rhythm compared to early risers. Connotations associated with 

<ipython-input-24-837785a48e1d>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['gpt_prompt_2_resp'].fillna(df['compound'], inplace=True)


#### Prompt 4: Alternative chain-of-thought

In [33]:
# GPT Prompt #4

import json
from google.colab import userdata
from openai import OpenAI

# initialize openai client with colab secret key
# client = OpenAI(api_key=userdata.get('OPENAI_PROJECT_KEY'))
client = OpenAI(api_key='sk-IQWpWGw3rqw1OEv102Cb0BHqya3ysEI9KKzxXNIdt3oDssnL',base_url="https://api.aiproxy.io/v1")

def generate_paraphrases_4(compound, sentence):
    """
    Generate paraphrases using GPT-4.
    """
    # Create a combined prompt
    # examples = "\n".join([
    #     f'Phrase: "{nc}"\nSentence: "{sent}"\n' for nc, sent in zip(compounds, sentences)
    # ])
    prompt = f"""You are a linguistics expert specializing in idioms. Given a potentially idiomatic phrase, and a sentence containing that phrase, you should determine whether the phrase is used literally or idiomatically in this particular instance, and describe the meaning of the phrase in this context.
Explain your reasoning.

Example #1
Phrase: "kick the bucket"
Sentence: "As John stood up from the table he accidentally kicked the bucket that was hidden underneath it."
Reasoning: To kick is to strike with the foot. A bucket is a container made of metal or plastic for carrying water. Kicking a bucket therefore means striking a container with the foot.
Alternatively, to kick the bucket, when used idiomatically, can mean to die.
Which option makes more sense in this case?
In the literal case, the sentence would mean that John is kicking a physical object, which we are told is underneath the table. It is plausible that a bucket could be hidden underneath a table and that standing up would cause one to kick that bucket.
In the idiomatic case, the sentence would mean that John accidentally dies as he stands up from the table, but then what is hidden underneath the table? The action of dying cannot be underneath a table. This usage does not make sense.
Therefore, the phrase is used literally in this instance.
Example 1 response:
{{
  "phrase": "kick the bucket",
  "usage": "literal",
  "meaning": "kicking a bucket, which might cause pain in the foot or cause a loud noise."
}}


Example #2
Idiom: "piece of cake"
Sentence: "Sarah thought the final exam was a piece of cake compared to the midterm."
Reasoning: A cake is a baked dessert, a piece of cake is a small amount of desert.
Alternatively, in idiomatic usage, something is considered a piece of cake if it is easy or requires little effort.
Which option makes more sense in this case?
In the literal case, the sentence would mean that Sarah thinks her exam is a piece of dessert. The piece of cake is compared to a different exam. It is unlikely that an exam is literally a piece of cake, and cake is not a comparable value like distance or size or intensity. This usage is unlikely.
In the idiomatic case, the sentence would mean that Sarah thinks her final exam was easy compared to the midterm. It is common to compare one exam with another in terms of its difficulty, so this usage makes sense.
Therefore, the phrase is used idiomatically in this instance.
Example 2 response:
{{
  "phrase": "piece of cake",
  "usage": "idiomatic",
  "meaning": "an activity or task that is delightfully easy, requiring little effort and quickly accomplished."
}}


Now, your given the following phrase and sentence:
Phrase: "{compound}"
Sentence: "{sentence}"

Response in the this format:
{{
  "phrase": <phrase>,
  "usage": <literal or idiomatic>,
  "meaning": <the meaning of the phrase in the context of the given sentence>
}}
Now, your output:

"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
        )
        # pos = response.choices[0].message.content.strip().find('Example 3 response')
        # json_response = response.choices[0].message.content.strip()[pos+19:]
        json_response = response.choices[0].message.content.strip()
        content = json.loads(json_response)
        print(f"Idiom: {content['phrase']}\nMeaning: {content['meaning']}\nUsage: {content['usage']}")
        return content
    except Exception as e:
        print(f"Error generating paraphrases: {e}")
        return {}

In [ ]:
# Apply to the DataFrame in batches
results = []
for i in range(len(df)):
    batch = df.iloc[i]
    compound = batch['compound']
    sentence = batch['sentence']
    batch_results = generate_paraphrases_4(compound, sentence)
    results.append(batch_results)

# # Map results back to DataFrame
df['gpt_prompt_4_resp'] = [res['meaning'] for res in results]
df['gpt_prompt_4_type'] = [res['usage'] for res in results]

Idiom: elbow grease
Meaning: effort and hard work, particularly physical effort, put into a task or project to achieve a desired result.
Usage: idiomatic
Idiom: night owl
Meaning: a person who prefers to stay up late at night and is typically active during the evening hours.
Usage: idiomatic
Idiom: heart of gold
Meaning: a person who is genuinely kind, generous, and caring, despite outward appearances or circumstances.
Usage: idiomatic
Idiom: agony aunt
Meaning: a person or organization that gives advice and assistance, especially regarding problems or concerns, similar to how an agony aunt in a newspaper provides advice on personal issues.
Usage: idiomatic
Idiom: shrinking violet
Meaning: something that is bold and assertive in character or flavor, contrasting with the usual meaning of a 'shrinking violet' as a person who is shy or timid.
Usage: idiomatic
Idiom: green fingers
Meaning: a talent or natural ability for gardening and growing plants effectively, often indicated by the pres

In [ ]:
# with open('gpt_prompt_4_results.csv', mode="w", newline="", encoding="utf-8") as file:
#     fieldnames = results[0].keys()
#     writer = csv.DictWriter(file, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerows(results)

In [ ]:
df['gpt_prompt_4_correct_type'] = df['gpt_prompt_4_type'] == df['sentence_type']

In [ ]:
df['gpt_prompt_4_correct_type'].sum()

In [ ]:
df_prompt_4_resp = pd.read_csv('gpt_prompt_6_responses.csv')

In [ ]:
df['text_input_exp_4'] = df_prompt_4_resp['meaning']
df['gpt_prompt_4_usage'] = df_prompt_4_resp['usage']

#### GPT Prompt 3: NC in-context definition, prompt decomposition (subtasks)

In [ ]:
# GPT Prompt #3

import json
from google.colab import userdata
from openai import OpenAI

# initialize openai client with colab secret key
# client = OpenAI(api_key=userdata.get('OPENAI_PROJECT_KEY'))
client = OpenAI(api_key='sk-IQWpWGw3rqw1OEv102Cb0BHqya3ysEI9KKzxXNIdt3oDssnL',base_url="https://api.aiproxy.io/v1")

def generate_paraphrases_batched_3(compounds, sentences):
    """
    Generate paraphrases in batches using GPT-4.
    """
    # Create a combined prompt
    examples = "\n\n".join([
        f'Phrase: "{nc}"\nSentence: "{sentence}"' for nc, sentence in zip(compounds, sentences)
    ])

    prompt = f"""
You are a linguistics expert specializing in figurative language. You will be given a set samples, each containing a potentially figurative English phrase paired with a sentence that said phrase is used in.
For each sample, you are to do the following:
1. Read the sentence; consider how the phrase is used in the sentence. It might be used figuratively (i.e. as an idiom), and it might be used literally (i.e. word composition).
2. Verbose explanation: Given your familiarity with the phrase's possible meanings, and having considered how it's used in the sentence, give a verbose explanation of what the phrase means in the context of the sentence. This can be a few sentences long.
3. Determine usage: State whether the phrase is used figuratively or literally in the sentence.
4. Definition: A concise, generalized definition of the phrase in this sentence.
5. Other usage definition: Assuming the phrase has both a literal definition and a figurative definition, give the definition for the OTHER usage.

Example #1:
<Sample>
Phrase: "cold turkey"
Sentence: "John quit smoking cold turkey and never looked back, not that it was easy."
---
<Output>
#3 - "Figuratively"
#4 - "Abruptly quitting a habit, marked by discomfort and determination"
#5 - "A turkey, which is a type of bird, that is cold"

Example #2:
<Sample>
Phrase: "piece of cake"
Sentence: "The boy eyed the piece of cake from afar as the waitress approached from across the room."
---
<Output>
#3 - "Literally"
#4 - "A slice of a sweet, baked dessert"
#5 - "A task that can be completed with no difficulty; something easy to accomplish"

Respond in this format:
{{"samples": [
    {{"phrase": "cold turkey", "verbose_definition": "<see #2 above>", "usage": "<see #3 above>", "definition": "<see #4 above>", "other_definition": "<see #5 above>"}},
    {{"phrase": "piece of cake", "verbose_definition": "<see #2 above>", "usage": "<see #3 above>", "definition": "<see #4 above>", "other_definition": "<see #5 above>" }}
]}}

In your definitions, do not preface your definition with phrases like, "The literal meaning would be..." or "The figurative meaning is...". Just give the definition, such that it could be used in a downstream task.
These are the samples:
{examples}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
        )
        content = json.loads(response.choices[0].message.content.strip())
        print(json.dumps(content, indent=4))
        results = {item["phrase"]: item["definition"] for item in content["samples"]}
        # for item in content["samples"]:
        #     print(f"Phrase: {item['phrase']}\nVerbose: {item['verbose_definition']}\nUsage: {item['usage']}\nDefinition: {item['definition']}\nOther Definition: {item['other_definition']}")
        return results
    except Exception as e:
        print(f"Error generating paraphrases: {e}")
        return {}

In [ ]:
# Apply to the DataFrame in batches
batch_size = 5
results = {}
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    compounds = batch['compound'].tolist()
    sentences = batch['sentence'].tolist()
    batch_results = generate_paraphrases_batched_3(compounds, sentences)
    results.update(batch_results)

# Map results back to DataFrame
df['gpt_prompt_3_resp'] = df['compound'].map(results)

In [ ]:
df[['compound', 'sentence_type', 'sentence', 'gpt_prompt_1_resp', 'gpt_prompt_2_resp', 'gpt_prompt_3_resp']].to_csv('prompt_responses.csv', index=False)

#### variations of prompt 3

In [ ]:
df['gpt_prompt_3_with_nc_for_literal'] = df.apply(
    lambda row: row['gpt_prompt_3_resp'] if row['sentence_type'] == 'literal' else row['compound'],
    axis=1
)

In [ ]:
df['gpt_prompt_3_with_sentence_for_literal'] = df.apply(
    lambda row: row['gpt_prompt_3_resp'] if row['sentence_type'] == 'literal' else row['compound'],
    axis=1
)

### Experiment and model configurations

#### Experiments config

In [ ]:
experiments = [
    {
        "name": "Baseline (Sentences)",
        "text_inputs": df['sentence']
    },
    {
        "name": "NC-Only",
        "text_inputs": df['compound']
    },
    {
        "name": "GPT Prompt 1",
        "text_inputs": df['gpt_prompt_1_resp']
    },
    {
        "name": "GPT Prompt 2",
        "text_inputs": df['gpt_prompt_2_resp']
    },
    {
        "name": "GPT Prompt 3",
        "text_inputs": df['gpt_prompt_3_resp']
    },
    {
        "name": "GPT Prompt 3 with nc for literal",
        "text_inputs": df['gpt_prompt_3_with_nc_for_literal']
    },
    {
        "name": "GPT Prompt 3 with sentence for literal",
        "text_inputs": df['gpt_prompt_3_with_sentence_for_literal']
    },
    {
        "name": "GPT Prompt 4 with sentence if predicted literal",
        "text_inputs": df['text_input_exp_4']
    }
]

#### Models config

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

openclip_model_version = "ViT-B-32"
model_openclip, _, preprocess_openclip = open_clip.create_model_and_transforms(openclip_model_version, pretrained='laion2b_s34b_b79k')
model_openclip.to(device)
model_openclip.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
from transformers import AlignProcessor, AlignModel

models = [
    {
        "base_model": "CLIP",
        "model_name": "ViT-B/32",
        "model": clip.load("ViT-B/32", device)[0],
        "preprocess": clip.load("ViT-B/32", device)[1]
    },
    {
        "base_model": "CLIP",
        "model_name": "ViT-L/14",
        "model": clip.load("ViT-L/14", device)[0],
        "preprocess": clip.load("ViT-L/14", device)[1]
    },
    {
        "base_model": "CLIP",
        "model_name": "RN50x64",
        "model": clip.load("RN50x64", device)[0],
        "preprocess": clip.load("RN50x64", device)[1]
    },
    {
        "base_model": "Align",
        "model_name": "Base",
        "model": AlignModel.from_pretrained("kakaobrain/align-base"),
        "preprocess": AlignProcessor.from_pretrained("kakaobrain/align-base")
    },
    {
        "base_model": "open_clip",
        "model_name": openclip_model_version,
        "model": model_openclip,
        "preprocess": preprocess_openclip
    }
]

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 53.4MiB/s]
100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 80.4MiB/s]
100%|█████████████████████████████████████| 1.26G/1.26G [02:23<00:00, 9.46MiB/s]


config.json:   0%|          | 0.00/5.25k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/690M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# openclip_model_version = "ViT-B-32"
# model_openclip, _, preprocess_openclip = open_clip.create_model_and_transforms(openclip_model_version, pretrained='laion2b_s34b_b79k')
# model_openclip.to(device)
# model_openclip.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active

# models.append({
#     "base_model": "open_clip",
#     "model_name": openclip_model_version,
#     "model": model_openclip,
#     "preprocess": preprocess_openclip
# })

### RUN ALL

In [ ]:
# UPDATED LOOP

# Initialize an empty list to store combined results
combined_predictions = []

# Run all experiments on all models
for exp in experiments:
    print(f"\nRunning \"{exp['name']}\" on {len(models)} models...")

    for idx, conf in enumerate(models):
        print(f"\nModel [{idx+1}/{len(models)}]: {conf['model_name']} ({conf['base_model']})")

        metrics, preds, confidence = run_experiment(
            model=conf['model'],
            processor=conf['preprocess'],
            df=df,
            image_paths=df['image_paths'],
            text_inputs=exp['text_inputs'],
            model_name=conf['model_name'],
            experiment_name=exp['name'],
            base_model=conf['base_model']
        )

        # Append predictions directly to combined_predictions
        save_predictions_to_df(
            df=df,
            image_paths=df['image_paths'],
            predictions=preds,
            confidence_scores=confidence,
            metrics=metrics,
            combined_predictions=combined_predictions,
            base_model=conf['base_model'],
            model_name=conf['model_name'],
            experiment_name=exp['name']
        )

# Convert the combined_predictions list into a DataFrame
combined_predictions_df = pd.DataFrame(combined_predictions)

# Save the final combined DataFrame to a CSV file
combined_predictions_df.to_csv('combined_predictions.csv', index=False)
print("Combined predictions saved to combined_predictions.csv")



Running "Baseline (Sentences)" on 5 models...

Model [1/5]: ViT-B/32 (CLIP)
Results saved to experiment_results.csv
Top-1 Accuracy: 45.71%
Average Spearman Correlation: 0.12
Average Weighted Accuracy: 0.35
Predictions added to combined DataFrame for model: ViT-B/32, experiment: Baseline (Sentences)

Model [2/5]: ViT-L/14 (CLIP)
Results saved to experiment_results.csv
Top-1 Accuracy: 47.14%
Average Spearman Correlation: 0.15
Average Weighted Accuracy: 0.36
Predictions added to combined DataFrame for model: ViT-L/14, experiment: Baseline (Sentences)

Model [3/5]: RN50x64 (CLIP)
Results saved to experiment_results.csv
Top-1 Accuracy: 50.00%
Average Spearman Correlation: 0.18
Average Weighted Accuracy: 0.37
Predictions added to combined DataFrame for model: RN50x64, experiment: Baseline (Sentences)

Model [4/5]: Base (Align)
Results saved to experiment_results.csv
Top-1 Accuracy: 48.57%
Average Spearman Correlation: 0.15
Average Weighted Accuracy: 0.39
Predictions added to combined DataFr

### Download files

In [ ]:
!zip -r results.zip experiment_results.csv combined_predictions.csv
from google.colab import files
files.download('results.zip')


  adding: experiment_results.csv (deflated 82%)
  adding: combined_predictions.csv (deflated 90%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>